# 🚢 Titanic Survival Prediction using KNN
---
This notebook explores the Titanic dataset, performs **data preprocessing** and **feature engineering**, and applies a **K-Nearest Neighbors (KNN)** classifier to predict passenger survival.
We'll visualize key patterns, clean data, and tune our KNN model for best accuracy.

In [ ]:

# 1️⃣ Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Optional: set style
sns.set(style="whitegrid")

# 2️⃣ Load Dataset
df = pd.read_csv("titanic.csv")
df.head()


In [ ]:

# Basic info about dataset
df.info()


In [ ]:

# Check missing values visually
plt.figure(figsize=(8,5))
sns.heatmap(df.isnull(), cbar=False, cmap="viridis")
plt.title("Missing Values Heatmap")
plt.show()


In [ ]:

# 3️⃣ Some quick visual insights
plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
sns.countplot(x='Sex', hue='Survived', data=df, palette='pastel')
plt.title('Survival by Gender')

plt.subplot(1,3,2)
sns.countplot(x='Pclass', hue='Survived', data=df, palette='cool')
plt.title('Survival by Class')

plt.subplot(1,3,3)
sns.histplot(df['Age'].dropna(), kde=True, color='teal')
plt.title('Age Distribution')

plt.tight_layout()
plt.show()


## 🔧 Feature Engineering
We'll extract useful information such as passenger titles, family size, and whether they were alone. We’ll also fill missing values and encode categorical data.

In [ ]:

# Select relevant columns
df = df[['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Name','Cabin','Ticket']]

# Title extraction
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df['Title'] = df['Title'].replace(['Mlle','Ms'],'Miss')
df['Title'] = df['Title'].replace(['Mme'],'Mrs')
df['Title'] = df['Title'].replace(['Countess','Lady','Sir','Don','Jonkheer','Major','Col','Capt','Rev','Dr'],'Rare')

# Family features
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

# Cabin and ticket features
df['HasCabin'] = df['Cabin'].apply(lambda x: 0 if pd.isna(x) else 1)
df['Ticket_Len'] = df['Ticket'].apply(lambda x: len(str(x)))
df['Ticket_Prefix'] = df['Ticket'].apply(lambda x: str(x)[0])

# Fill missing values
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Fare'].fillna(df['Fare'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Label encoding
label = LabelEncoder()
for col in ['Sex','Embarked','Title','Ticket_Prefix']:
    df[col] = label.fit_transform(df[col])

# Drop unused columns
df.drop(['Name','Cabin','Ticket'], axis=1, inplace=True)

df.head()


In [ ]:

# 4️⃣ Split Data
X = df.drop('Survived', axis=1)
y = df['Survived']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:

# 5️⃣ Train and Tune KNN
param_grid = {'n_neighbors': np.arange(1, 31)}
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid, cv=5)
grid.fit(X_train, y_train)

best_k = grid.best_params_['n_neighbors']
print("✅ Best K:", best_k)

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("🎯 Final KNN Accuracy:", round(acc * 100, 2), "%")

